In [1]:
#Импортирование библиотек
from Timage import Timage
import numpy as np
import scipy.io as sc
from scipy.signal import convolve2d
from PIL import Image

m = sc.loadmat('resources/sector_one.mat')

#Берет n-ое изображение из sector_one.mat
def get_timage(n):
    global m
    return Timage(array=np.array([[m['A'][j][i][n] for j in range(416)] for i in range(442)], dtype=np.float32))

In [2]:
class DefectMap:
    def __init__(self, array):
        self.array = array
    
    def direct(self, direction, contrast_level, pallete):
        np_pallete = np.array(pallete, dtype=np.float32)
        
        directed = 128 - np.real(self.array * np.conjugate(direction)) 

        mean = np.mean(directed)
        f = lambda x: (x - mean) / (1 - contrast_level) + mean
        contrasted = directed.copy()

        m, n = len(directed), len(directed[0])
        for i in range(m):
            for j in range(n):
                contrasted[i][j] = min(255, max(0,   f(float(directed[i][j]))  ))

        colored = np.multiply.outer(contrasted, np_pallete[1]/255) + np.multiply.outer(255-contrasted, np_pallete[0]/255)
        return colored

In [4]:
from tqdm import trange
from math import sin, cos, pi
t60 = get_timage(60)
def_map = DefectMap(array=t60.defect_map(radius=3, stddev=1.5))
num_pictures = 1200
# 1200 -> 14min, 6min
for i in trange(num_pictures):
    phi = i * pi / (num_pictures - 1) # [0, pi]
    im = Image.fromarray(def_map.direct(np.complex64(complex(cos(phi), sin(phi))), contrast_level=0.998, pallete=[[0,0,0],[255,255,255]]).astype('uint8'))
    im.save(f'temp/{i}.png')

  5%|▍         | 59/1200 [00:19<06:24,  2.97it/s]


KeyboardInterrupt: 

In [2]:
from tqdm import trange
for i in trange(1000):
    get_timage(i).defect_map(radius=3, stddev=1.5, contrast_level=0.998, direction=np.complex64(complex(2**(-0.5), 2**(-0.5))), pallete=np.array([[0,0,0],[255,255,255]], dtype=np.float32)).save(f'temp/{i}.png')

  2%|▏         | 19/1000 [00:07<06:34,  2.48it/s]


KeyboardInterrupt: 

In [ ]:
from moviepy import ImageSequenceClip

output_video = "60rotate_result1200.mp4"
fps = 240

image_files = [f'60rotate_meme//{i}.png' for i in range(1200)]

clip = ImageSequenceClip(image_files, fps=fps)

clip.write_videofile(output_video, audio=False)

MoviePy - Building video 60rotate_result1200.mp4.
MoviePy - Writing video 60rotate_result1200.mp4



MoviePy - Done !
MoviePy - video ready 60rotate_result1200.mp4


In [4]:
t60 = get_timage(60)
t60.defect_map(radius=3, stddev=1.5, contrast_level=0.998).show()